## MATH/STAT 4450/8456 Contest3

Description of variables  
• recordID: Row ID.  
• orderID: Order ID. Different rows may have the same order ID.  
• orderDate: Order date.  
• itemID: Item ID. One order may have multiple items.  
• colorCode: Color code of the item.  
• sizeCode: Size code of the item.  
• typeCode: Type code of the item.  
• price: Price of the item.  
• recommendedPrice: Recommended retail price.  
• voucherID: Voucher ID.  
• voucherAmount: Voucher value PER ORDER.  
• customerID: Customer ID.  
• deviceCode: Device type.  
• paymentCode: Payment type.  
• return: If the item is returned or not. 1: yes. 0: no.

### Step 1: Data Preprocessing

In [1]:
# Import required libraries
from DataPreparation import DataPrep
import pandas as pd
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, mean_squared_error
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split


In [2]:
#Load data
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
all = pd.concat((train, test), sort=False)
#Data clean and feature engineering
all_cleaned = DataPrep().run(all)
all_cleaned.fillna(0, inplace=True)
len_train = len(train)
train_df = all_cleaned[0:len_train]
test_df = all_cleaned[len_train:]

## Step: feature selection

In [3]:
# Calculate the correlation matrix
correlation_matrix = train_df.corr()
correlation_values = correlation_matrix['return'].abs().sort_values(ascending=False)
top_correlated_features = correlation_values[correlation_values.index != 'return'].head(12)
top_correlated_feature_names = top_correlated_features.index.tolist()

/var/folders/85/3hk_g8w922zfg2gmpw407zj00000gn/T/ipykernel_46860/2006090936.py:2: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  correlation_matrix = train_df.corr()


## Step: Validation Approach

(ii) Data filtering -> Data cleaning -> Validation set approach (70-30 ratio, random sampling) -> Model Fitting -> Report the validation error.

In [6]:
X = train_df[top_correlated_feature_names]
y = train_df['return']

# Validation set approach
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=42)
xgb_model = XGBClassifier(max_depth = 8, 
                        min_child_weight= 3,
                        objective='binary:logistic',
                        nthread=4,
                        scale_pos_weight=1,
                        n_estimators = 200,
                        learning_rate=0.1,
                        gamma=0, 
                        subsample=0.8, 
                        colsample_bytree=0.8
                        )
xgb_model.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=10)
y_pred = xgb_model.predict(X_val)
accuracy = round(accuracy_score(y_val, y_pred),4)
val_error = round(mean_squared_error(y_val, y_pred),4)
print(f"Accuracy on the validation set: {accuracy} ")
print(f"Validation error: {val_error}")


[0]	validation_0-logloss:0.68401
[10]	validation_0-logloss:0.64333
[20]	validation_0-logloss:0.63540
[30]	validation_0-logloss:0.63269
[40]	validation_0-logloss:0.63150
[50]	validation_0-logloss:0.63061
[60]	validation_0-logloss:0.63008
[70]	validation_0-logloss:0.62937
[80]	validation_0-logloss:0.62899
[90]	validation_0-logloss:0.62830
[100]	validation_0-logloss:0.62798
[110]	validation_0-logloss:0.62774
[120]	validation_0-logloss:0.62740
[130]	validation_0-logloss:0.62710
[140]	validation_0-logloss:0.62686
[150]	validation_0-logloss:0.62670
[160]	validation_0-logloss:0.62655
[170]	validation_0-logloss:0.62634
[180]	validation_0-logloss:0.62622
[190]	validation_0-logloss:0.62612
[199]	validation_0-logloss:0.62597
Accuracy on the validation set: 0.6487 
Validation error: 0.3513


## Step: Make prediction on test

In [5]:
test_df = test_df[top_correlated_feature_names]
xgb_pred = xgb_model.predict(test_df)
df_submission = pd.read_csv("sample_submission.csv")
df_submission['return'] = xgb_pred
print(df_submission['return'].value_counts())
df_submission.to_csv('submission_xgb0.6487.csv', index=False)

1    174714
0     97022
Name: return, dtype: int64
